In [1]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.10:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.7/dist-packages/kaggle-1.5.10.dist-info/*
    /usr/local/lib/python3.7/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.10
     |████████████████████████████████| 1.5MB 5.7MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=808e6f69d6ba6490e17e09fb53ae3bdee09626fdbee06a8ca481a3f7c6a5dc30
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
-rw-r--r-- 1 root root 65 Mar 17 06:25 kaggle.json


In [2]:
!kaggle competitions download -c sejong-ai-challenge-p2
!unzip sejong-ai-challenge-p2.zip

 57% 17.0M/29.8M [00:00<00:01, 12.0MB/s]
100% 29.8M/29.8M [00:01<00:00, 31.0MB/s]


In [18]:
!pip install fasttext

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

train = shuffle(train, random_state=0)
train['Text'] = train['Text'].astype(str)
test['Text'] = test['Text'].astype(str)

train

,Unnamed: 0,Text,Label
71019,64872,Good product. It has a slightly plastic feel ...,1
151264,207111,it worked on my phone but it didn't come with ...,0
126784,77813,We use this for my teenage daughters phone and...,1
98978,200321,Great !!!,1
58578,148107,I love collecting phone cases because I like t...,1
...,...,...,...
152315,260160,Did not like.,0
176963,49409,"I like the overall design of this case a lot, ...",0
117952,10677,Breaks easily. Mine didn't even last a week,0
173685,167021,I had the phone for a month and absolutely lov...,0


In [2]:
train['label_ft'] = '__label__' + train['Label'].astype(str)

In [3]:
train

,Unnamed: 0,Text,Label,label_ft
71019,64872,Good product. It has a slightly plastic feel ...,1,__label__1
151264,207111,it worked on my phone but it didn't come with ...,0,__label__0
126784,77813,We use this for my teenage daughters phone and...,1,__label__1
98978,200321,Great !!!,1,__label__1
58578,148107,I love collecting phone cases because I like t...,1,__label__1
...,...,...,...,...
152315,260160,Did not like.,0,__label__0
176963,49409,"I like the overall design of this case a lot, ...",0,__label__0
117952,10677,Breaks easily. Mine didn't even last a week,0,__label__0
173685,167021,I had the phone for a month and absolutely lov...,0,__label__0


In [4]:
for i in test.index:
    test['Text'][i] =test['Text'][i].replace('\n', '')

for i in train.index:
    train['Text'][i] = train['Text'][i].replace('\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import fasttext
models = []
train_scores = []
pred_list= []
val_scores = []

skf = StratifiedKFold(n_splits = 3, shuffle=True)

for train_index, test_index in skf.split(train['Text'], train['label_ft']):
    train[['Text', 'label_ft']].iloc[train_index].to_csv('fttrain.csv', index=None, header=None)
    model = fasttext.train_supervised('fttrain.csv', epoch=15, wordNgrams=3, minCount=2, maxn=5, verbose=0)
    models.append(model)

    train_pred = [model.predict(x)[0][0].split('__')[-1] for x in train.iloc[train_index]['Text']]
    train_score = f1_score(train['Label'].values[train_index].astype(str), train_pred, average='macro')
    print('Train score: ', train_score)
    train_scores.append(train_score)

    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train.iloc[test_index]['Text']]
    val_score = f1_score(train['Label'].values[test_index].astype(str), val_pred, average='macro')
    print('Val score: ', val_score)
    val_scores.append(val_score)

print('mean train score: ', np.mean(train_scores))
print('mean val score: ', np.mean(val_scores))

In [ ]:
test['Text'].loc[7]

"See if you sound like me: - Large family with multiple devices that need charging either by Apple cables or Micro USB - Cables are always getting lost, eaten, frayed/broken, lent, or sucked into another dimensionI'm usually on Ebay or Amazon once every 3 months to buy a wholesale pack of micro USB or Apple cables because the 9+ devices we charge at home go through cables like most families go through milk.  Most cables I buy do not stand up to our abuse, and the usual break point is the cable to micro USB head.These cables from Pro Rich seem to be different.  The break point looks to be more reinforced, the cables themselves have some heft and sturdiness to them, and the LENGTH(!) is outstanding.  I would recommend these to anyone who feels like a wholesale dealer of charging cables (or anyone that seems to go through them on a regular basis)."

In [36]:

for i in test.index:
    test_pred = text_clf_model.predict(test['Text'][i], k=1)[0][0]
    submission['Label'][i] =  test_pred.split('__')[-1]


print(submission)

          id  Label
0          0      1
1          1      0
2          2      0
3          3      0
4          4      0
...      ...    ...
83199  83199      0
83200  83200      0
83201  83201      0
83202  83202      0
83203  83203      1

[83204 rows x 2 columns]


In [37]:
submission.to_csv('submission1.csv', index=None, header=True)

In [38]:
!kaggle competitions submit -c sejong-ai-challenge-p2 -f submission1.csv -m "Message"

100% 639k/639k [00:08<00:00, 77.5kB/s]
Successfully submitted to Sejong AI Challenge 문제2